In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor

In [3]:
train=pd.read_csv('train/train.csv')
test=pd.read_csv('test/test.csv')

data=pd.concat([train,test])

In [4]:
data.head()

,Age,City_Category,Gender,Marital_Status,Occupation,Product_Category_1,Product_Category_2,Product_Category_3,Product_ID,Purchase,Stay_In_Current_City_Years,User_ID
0,0-17,A,F,0,10,3,NaN,NaN,P00069042,8370.0,2,1000001
1,0-17,A,F,0,10,1,6.0,14.0,P00248942,15200.0,2,1000001
2,0-17,A,F,0,10,12,NaN,NaN,P00087842,1422.0,2,1000001
3,0-17,A,F,0,10,12,14.0,NaN,P00085442,1057.0,2,1000001
4,55+,C,M,0,16,8,NaN,NaN,P00285442,7969.0,4+,1000002


In [5]:
data.Gender.value_counts()

M    590031
F    193636
Name: Gender, dtype: int64

In [6]:
train.info(),test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550068 entries, 0 to 550067
Data columns (total 12 columns):
User_ID                       550068 non-null int64
Product_ID                    550068 non-null object
Gender                        550068 non-null object
Age                           550068 non-null object
Occupation                    550068 non-null int64
City_Category                 550068 non-null object
Stay_In_Current_City_Years    550068 non-null object
Marital_Status                550068 non-null int64
Product_Category_1            550068 non-null int64
Product_Category_2            376430 non-null float64
Product_Category_3            166821 non-null float64
Purchase                      550068 non-null int64
dtypes: float64(2), int64(5), object(5)
memory usage: 50.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233599 entries, 0 to 233598
Data columns (total 11 columns):
User_ID                       233599 non-null int64
Product_ID                    

(None, None)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 783667 entries, 0 to 233598
Data columns (total 12 columns):
Age                           783667 non-null object
City_Category                 783667 non-null object
Gender                        783667 non-null object
Marital_Status                783667 non-null int64
Occupation                    783667 non-null int64
Product_Category_1            783667 non-null int64
Product_Category_2            537685 non-null float64
Product_Category_3            237858 non-null float64
Product_ID                    783667 non-null object
Purchase                      550068 non-null float64
Stay_In_Current_City_Years    783667 non-null object
User_ID                       783667 non-null int64
dtypes: float64(3), int64(4), object(5)
memory usage: 77.7+ MB


In [8]:
train.Product_Category_1.value_counts(),test.Product_Category_1.value_counts()

(5     150933
 1     140378
 8     113925
 11     24287
 2      23864
 6      20466
 3      20213
 4      11753
 16      9828
 15      6290
 13      5549
 10      5125
 12      3947
 7       3721
 18      3125
 20      2550
 19      1603
 14      1523
 17       578
 9        410
 Name: Product_Category_1, dtype: int64, 5     65017
 1     60321
 8     48369
 2     10192
 11    10153
 6      8860
 3      8578
 4      5003
 16     4105
 15     2694
 13     2381
 10     2248
 12     1663
 7      1624
 18     1311
 14      663
 17      223
 9       194
 Name: Product_Category_1, dtype: int64)

In [9]:
data.User_ID.max(),data.User_ID.min()

(1006040, 1000001)

In [10]:
data.Stay_In_Current_City_Years.value_counts()

1     276425
2     145427
3     135428
4+    120671
0     105716
Name: Stay_In_Current_City_Years, dtype: int64

In [11]:
dataP=pd.DataFrame()
dataP=pd.concat([pd.get_dummies(data['Product_Category_1'].fillna(0).astype(int)),
                 pd.get_dummies(data['Product_Category_2'].fillna(0).astype(int)),
                 pd.get_dummies(data['Product_Category_3'].fillna(0).astype(int))],axis=1)
dataP=dataP.groupby(dataP.columns,axis=1).sum().drop(0,axis=1)

In [12]:
dataP[:5]

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
age={'0-17':1,'18-25':2,'26-35':3,'36-45':4,'46-50':5,'51-55':6,'55+':7}

In [14]:
dataB=pd.DataFrame()
#dataB=pd.concat([pd.get_dummies(data['Age']),
dataB=pd.get_dummies(data['City_Category'])
 #                dataP],axis=1)
dataB['Age']=data['Age'].map(age)
dataB['Cat_1']=data['Product_Category_1'].fillna(0).astype(int)
dataB['Cat_2']=data['Product_Category_2'].fillna(0).astype(int)
dataB['Cat_3']=data['Product_Category_3'].fillna(0).astype(int)
dataB['Stay']=data.Stay_In_Current_City_Years.replace('4+',4)
dataB['Gender']=data.Gender.map({'M':1,'F':0})
dataB['Marital_Status']=data.Marital_Status
dataB['Occupation']=data['Occupation']
dataB['UserID']=data['User_ID']-1000000
dataB['PID']=data['Product_ID'].str.strip('P')
dataB['Purchase']=data['Purchase']

In [15]:
dataB.head()

,A,B,C,Age,Cat_1,Cat_2,Cat_3,Stay,Gender,Marital_Status,Occupation,UserID,PID,Purchase
0,1,0,0,1,3,0,0,2,0,0,10,1,00069042,8370.0
1,1,0,0,1,1,6,14,2,0,0,10,1,00248942,15200.0
2,1,0,0,1,12,0,0,2,0,0,10,1,00087842,1422.0
3,1,0,0,1,12,14,0,2,0,0,10,1,00085442,1057.0
4,0,0,1,7,8,0,0,4,1,0,16,2,00285442,7969.0


In [16]:
train_w=dataB[dataB['Purchase'].notnull()]
test_w=dataB[dataB['Purchase'].isnull()].drop('Purchase',axis=1)
train.shape,test.shape

((550068, 12), (233599, 11))

In [17]:
X_train,X_test,y_train,y_test=train_test_split(train_w.drop('Purchase',axis=1),train_w.Purchase)

In [19]:
md=GradientBoostingRegressor(min_samples_split=8000,max_depth=11,max_features='sqrt',n_estimators=500).fit(X_train,y_train)

print('train score: {}'.format(md.score(X_train,y_train)))
print('test score: {}'.format(md.score(X_test,y_test)))

train_pred=md.predict(X_train)
test_pred=md.predict(X_test)
rms_train=math.sqrt(mean_squared_error(y_train,train_pred))
rms_test=math.sqrt(mean_squared_error(y_test,test_pred))
print('train set RMS score: {}'.format(rms_train))
print('test set RMS score: {}'.format(rms_test))

train score: 0.7324015783376772
test score: 0.7221481667371086
train set RMS score: 2596.552104370814
test set RMS score: 2653.441891320895


In [18]:
params={'max_depth':range(5,16,2), 'min_samples_split':range(1000,10000,1000)}
grid=GridSearchCV(GradientBoostingRegressor(max_features='sqrt',n_estimators=500),params,cv=5,n_jobs=-1)
grid.fit(X_train,y_train)
print('best parameters: {}'.format(grid.best_params_))
print('mean train score: {}'.format(grid.cv_results_['mean_train_score']))
print('mean test score: {}'.format(grid.cv_results_['mean_test_score']))

best parameters: {'max_depth': 15, 'min_samples_split': 1000}
mean train score: [ 0.70550019  0.70331976  0.70134645  0.69890608  0.69829979  0.69708041
  0.69699467  0.69589971  0.69446316  0.73362284  0.72738207  0.72338167
  0.71970553  0.71672275  0.715004    0.71330653  0.71146885  0.71043269
  0.75696088  0.74552839  0.73870214  0.73427278  0.73023839  0.72742305
  0.72454962  0.72248844  0.72072021  0.77473727  0.75951276  0.75022639
  0.74446884  0.73993862  0.73667566  0.73344756  0.73004185  0.72769224
  0.79008784  0.77036829  0.75972059  0.75263896  0.74720215  0.74273851
  0.73906653  0.73656422  0.73313773  0.80305404  0.77905406  0.76679664
  0.75843197  0.75227876  0.74681965  0.74316888  0.73953132  0.73681099]
mean test score: [ 0.70002356  0.69846828  0.69669656  0.69468178  0.6942084   0.69320624
  0.69323195  0.69231143  0.6911982   0.71970209  0.71632533  0.7136334
  0.71087749  0.70886941  0.70759028  0.7061881   0.70518465  0.70445494
  0.73175289  0.72657687  0

/root/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


In [20]:
md=GradientBoostingRegressor(min_samples_split=8000,max_depth=11,max_features='sqrt',n_estimators=500).fit(train_w.drop('Purchase',axis=1),train_w.Purchase)

In [22]:
pred=md.predict(test_w)

solution=pd.DataFrame(data={'User_ID':list(test.User_ID),
                            'Product_ID':list(test.Product_ID),
                            'Purchase':pred})
solution.to_csv('solutions.csv',index=False)